<a href="https://colab.research.google.com/github/lavanya9782/Final-project-Guvi/blob/main/finalproject_QAmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
#creating login to hugging face  so as to share our model to community

from huggingface_hub import notebook_login

notebook_login()


In [ ]:
#!pip install accelerate -U

In [ ]:
#install the Transformers library with PyTorch support
!pip install transformers[torch]

In [ ]:
#!pip install scikit-learn

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
#loading a smaller subset of the SQuAD dataset from the hugging face Datasets library
squad = squad.train_test_split(test_size=0.2)

In [ ]:
# view a sample of dataset
squad["train"][1]

{'id': '56d00beb234ae51400d9c2d3',
 'title': 'New_York_City',
 'context': 'New York\'s high rate of public transit use, over 200,000 daily cyclists as of 2014, and many pedestrian commuters make it the most energy-efficient major city in the United States. Walk and bicycle modes of travel account for 21% of all modes for trips in the city; nationally the rate for metro regions is about 8%. In both its 2011 and 2015 rankings, Walk Score named New York City the most walkable large city in the United States. Citibank sponsored the introduction of 10,000 public bicycles for the city\'s bike-share project in the summer of 2013. Research conducted by Quinnipiac University showed that a majority of New Yorkers support the initiative. New York City\'s numerical "in-season cycling indicator" of bicycling in the city hit an all-time high in 2013.',
 'question': 'How many daily bicycle riders are there in NYC?',
 'answers': {'text': ['200,000'], 'answer_start': [49]}}

In [ ]:
#using the Hugging Face Transformers library to load a pre-trained tokenizer for the
# RoBERTa model fine-tuned on the SQuAD 2.0 dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
#defining a preprocessing function for the data using the tokenizer
#the preprocessing function takes raw examples from the SQuAD dataset, tokenizes them,
#identifies the positions of the answers within the tokenized context,
#and returns the processed inputs with start and end positions of the answers.

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # To Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# applying a preprocessing function to the squad dataset  in batches, and also  removing certain columns
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# DefaultDataCollator is a class provided by the Transformers library that helps with this
#data processing like  before feeding it into the model(tasks such as tokenization, batching, and padding).
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import sklearn
from sklearn.model_selection import GridSearchCV
#load the pre-trained question answering model  RoBERTa fine-tuned on the SQuAD 2.0 dataset,
#allowing us to use it for question answering tasks without needing to train it from scratch.

model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
#Training process
#define the training arguments for the training process
training_args = TrainingArguments(
    output_dir="lavan", #directory where the trained model and associated files will be saved.
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True, # trained model should be pushed to the Hugging Face Hub after training.
)
#The trainer class is responsible for managing the training process
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
# the training process,  iterates over the training data for the specified number of epochs
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.532021
2,0.479400,0.563544
3,0.479400,0.615616


TrainOutput(global_step=750, training_loss=0.3987195943196615, metrics={'train_runtime': 964.4887, 'train_samples_per_second': 12.442, 'train_steps_per_second': 0.778, 'total_flos': 2351670810624000.0, 'train_loss': 0.3987195943196615, 'epoch': 3.0})

In [ ]:
#push your trained model to the Hugging Face Model Hub.
trainer.push_to_hub( "LavanyaM/lavan")

CommitInfo(commit_url='https://huggingface.co/LavanyaM/lavan/commit/a6d69cbdb1c0280acb896074f3bf3a419bc815e0', commit_message='LavanyaM/lavan', commit_description='', oid='a6d69cbdb1c0280acb896074f3bf3a419bc815e0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# for  prediction - question and context
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
# for inference
from transformers import pipeline

question_answerer = pipeline("question-answering", model="lavan")
question_answerer(question=question, context=context)

{'score': 0.9969184398651123, 'start': 93, 'end': 95, 'answer': '13'}

In [ ]:
from sklearn.metrics import accuracy_score
from datasets import load_metric

# Load the F1 metric


In [ ]:
# evaluation
from sklearn.metrics import accuracy_score
from datasets import load_metric
from transformers import pipeline

# Load the question answering pipeline
question_answerer = pipeline("question-answering", model="lavan")

# Load the SQuAD metric for evaluation
squad_metric = load_metric("squad")

# Define the evaluation dataset
evaluation_data = [
    {"question": "How many programming languages does BLOOM support?", "context": "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages.", "answers": {"text": ["13"], "answer_start": [109]}},
    # Add more evaluation data as needed
]

# Generate predictions for the evaluation dataset
predictions = [{"prediction_text": question_answerer(example["question"], example["context"])["answer"]} for example in evaluation_data]

# Prepare references (ground truth answers)
references = [{"answers": example["answers"]} for example in evaluation_data]

# Compute evaluation metrics
evaluation_results = squad_metric.compute(predictions=predictions, references=references)

# Extract F1 score and accuracy from evaluation results
f1_score = evaluation_results["f1"]
accuracy = accuracy_score([example["answers"]["text"][0] for example in evaluation_data], [prediction["prediction_text"] for prediction in predictions])

print("F1 Score:", f1_score)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


F1 Score: 100.0
Accuracy: 1.0
